In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Install transformer and peft if you don't have it installed
# ! pip install transformers==4.31.0
# ! pip install peft
# ! pip install accelerate
# ! pip install bitsandbytes

This is a standalone notebook to train the NSQL model on a single GPU (e.g., A5000 with 24GB) with int8 and LoRA.

# Load the model

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("NumbersStation/nsql-llama-2-7B")
model = AutoModelForCausalLM.from_pretrained("NumbersStation/nsql-llama-2-7B", load_in_8bit=True, torch_dtype=torch.bfloat16, device_map='auto')

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

# Prepare the data

We use NumbersStation/NSText2SQL dataset as an example here and feel free to customize the training data based on your need.

In [4]:
from datasets import load_dataset
from torch.utils.data import Dataset
import copy

class NSText2SQLDataset(Dataset):
    def __init__(self, size=None, max_seq_length=2048):
        self.dataset = load_dataset("NumbersStation/NSText2SQL",split="train")
        if size:
            self.dataset = self.dataset.select(range(size))
        self.max_seq_length = max_seq_length

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        instruction = torch.tensor(tokenizer.encode(self.dataset[index]['instruction']), dtype=torch.int64)
        example = self.dataset[index]['instruction'] + self.dataset[index]["output"]
        example = tokenizer.encode(example)
        example.append(tokenizer.eos_token_id)
        padding = self.max_seq_length - len(example)
        example = torch.tensor(example, dtype=torch.int64)

        if padding < 0:
            example = example[:self.max_seq_length]
        else:
            example = torch.cat((example, torch.zeros(padding, dtype=torch.int64)))
            
        labels = copy.deepcopy(example)
        labels[: len(instruction)] = -100
        
        return {"input_ids": example, "labels": labels}

In [5]:
dataset = NSText2SQLDataset(size=1000, max_seq_length=1024)

Found cached dataset json (/data/senwu/data/huggingface/datasets/NumbersStation___json/NumbersStation--NSText2SQL-2ee760d614b948c2/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e)


# Prepare PEFT for model training

In [6]:
from peft import (
    get_peft_model,
    LoraConfig,
    TaskType,
    prepare_model_for_int8_training,
)


model.train()

model = prepare_model_for_int8_training(model)

lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules = ["q_proj", "v_proj"]
)

model = get_peft_model(model, lora_config)

# Finetune the model with Huggingface trainer

In [7]:
from transformers import default_data_collator, Trainer, TrainingArguments

output_dir = "training_run"

config = {
    'lora_config': lora_config,
    'learning_rate': 1e-4,
    'num_train_epochs': 1,
    'gradient_accumulation_steps': 2,
    'gradient_checkpointing': False,
}


training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    bf16=True,
    # logging strategies
    logging_dir=f"{output_dir}/logs",
    logging_strategy="steps",
    logging_steps=5,
    optim="adamw_torch_fused",
    **{k:v for k,v in config.items() if k != 'lora_config'}
)

In [8]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=default_data_collator,
)

# Start training
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/home/senwu/repos/nslabs/.venv/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
5,18.636100
10,13.724200
15,10.669200
20,8.244300
25,6.646400
30,5.631100
35,5.023000
40,4.750200
45,4.641000
50,4.547600


TrainOutput(global_step=62, training_loss=7.540836887974893, metrics={'train_runtime': 974.9212, 'train_samples_per_second': 1.026, 'train_steps_per_second': 0.064, 'total_flos': 4.029631866863616e+16, 'train_loss': 7.540836887974893, 'epoch': 0.99})

# Save model checkpoint

In [9]:
model.save_pretrained(output_dir)

# Evaluate the finetuned model

In [10]:
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=2)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear8bitLt(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): Dropout(p=0.05, inplace=False)
                (lora_A): Linear(in_features=4096, out_features=8, bias=False)
                (lora_B): Linear(in_features=8, out_features=4096, bias=False)
              )
              (k_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
              (v_proj): Linear8bitLt(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): Dropout(p=0.05, inplace=False)
                (lora_A): Linear(in_features=4096, out_features=8, bias=False)
                (lora_B): Linear(in_features=8, o

In [11]:
text = """CREATE TABLE stadium (
    stadium_id number,
    location text,
    name text,
    capacity number,
)

-- Using valid SQLite, answer the following questions for the tables provided above.

-- how many stadiums in total?

SELECT"""

model_input = tokenizer(text, return_tensors="pt").to("cuda")

generated_ids = model.generate(**model_input, max_new_tokens=100)
print(tokenizer.decode(generated_ids[0], skip_special_tokens=True))

CREATE TABLE stadium (
    stadium_id number,
    location text,
    name text,
    capacity number,
)

-- Using valid SQLite, answer the following questions for the tables provided above.

-- how many stadiums in total?

SELECT COUNT(*) FROM stadium
